In [ ]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import scanpy as sc
import anndata as ad
import numpy as np
import spagrn
from spagrn.network import Network  # Import the Network class
from spagrn import plot as prn
from pyscenic.aucell import aucell
from scipy.spatial.distance import jensenshannon



# Load data
PDAC = ad.read_h5ad('/Users/linliu/Library/CloudStorage/Box-Box/linliu/PDAC/ad_PDAC/PDAC_SIMVI.h5ad')
print("Initial PDAC shape:", PDAC.shape)
print("Initial PDAC.obs_names:", PDAC.obs_names[:5])  # Show first 5 for brevity

# Preprocess data
PDAC.X = PDAC.layers['counts']
PDAC.layers['counts'] = PDAC.layers['counts'].tocsc()
PDAC.obs['total_counts'] = np.sum(PDAC.layers['counts'], axis=1)
sc.pp.normalize_total(PDAC, target_sum=1e4)
sc.pp.log1p(PDAC)
PDAC = Network.preprocess(PDAC, min_genes=10, min_cells=20, min_counts=20, max_gene_num=4000)
print("Post-preprocessing shape:", PDAC.shape)
print("Post-preprocessing obs_names:", PDAC.obs_names[:5])


# Load input files
tfs_fn = r'/Users/linliu/Library/CloudStorage/OneDrive-AugustaUniversity/GRN Analysis Input Resources/allTFs_hg38.txt'
database_fn = r'/Users/linliu/Library/CloudStorage/OneDrive-AugustaUniversity/GRN Analysis Input Resources/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather'
motif_anno_fn = r'/Users/linliu/Library/CloudStorage/OneDrive-AugustaUniversity/GRN Analysis Input Resources/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl'
niches = pd.read_csv(r'/Users/linliu/Library/CloudStorage/OneDrive-AugustaUniversity/GRN Analysis Input Resources/lr_network_human.csv')


# Initialize InferNetwork
grn = spagrn.regulatory_network.InferNetwork(adata=PDAC, project_name="PDAC_project")
grn.add_params({'num_worker':12, 'auc_threshold': 0.05})
print("grn._data.to_df() index:", grn._data.to_df().index[:5])

# Verify index alignment
if not (grn._data.to_df().index.equals(PDAC.obs_names)):
    raise ValueError("Index mismatch between grn._data.to_df().index and PDAC.obs_names")

# Clear cache files
cache_files = ['project_adj.csv', 'project_auc.csv', 'project_motifs.csv', 
               'project_filtered_targets_receptor.json', 'project_spagrn.h5ad']
for f in cache_files:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed cache file: {f}")

# Run inference
try:
    grn.infer(
        database_fn,
        motif_anno_fn,
        tfs_fn,
        niche_df=niches,
        num_workers=1,
        cache=False,
        save_tmp=True,
        latent_obsm_key='spatial',
        model='bernoulli',
        n_neighbors=30,
        cluster_label='subleiden',
        layers='counts',
        umi_counts_obs_key="total_counts",
        batch_key='patient'
    )
    
    # Verify output
    print("Output adata.obs_names:", PDAC.obs_names[:5])
    print("Output auc_mtx index:", PDAC.obsm['auc_mtx'].index[:5])
    print("Successfully completed GRN inference!")
    
except Exception as e:
    print(f"Error during inference: {str(e)}")
    print(f"Error type: {type(e).__name__}")
    raise

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initial PDAC shape: (417793, 965)
Initial PDAC.obs_names: Index(['B10_1_1', 'B10_2_1', 'B10_3_1', 'B10_4_1', 'B10_5_1'], dtype='object')
Post-preprocessing shape: (417793, 965)
Post-preprocessing obs_names: Index(['B10_1_1', 'B10_2_1', 'B10_3_1', 'B10_4_1', 'B10_5_1'], dtype='object')
grn._data.to_df() index: Index(['B10_1_1', 'B10_2_1', 'B10_3_1', 'B10_4_1', 'B10_5_1'], dtype='object')
----------------------------------------
Project name is PDAC_project
Saving output files into /Users/linliu/miniconda3/envs/newspagrn/lib/python3.8/site-packages/spagrn
Saving temporary files to /Users/linliu/miniconda3/envs/newspagrn/lib/python3.8/site-packages/spagrn/tmp_files
----------------------------------------


Computing autocorrelations:   2%|▏         | 17/965 [54:47<41:27:20, 157.43s/it]

In [ ]:
rss = Network.regulon_specificity_scores(PDAC.obsm['auc_mtx'], PDAC.obs['annotation'])
    PDAC.uns['rss'] = rss  # Store RSS in PDAC.uns['rss']